In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import torch
import torch.nn as nn
print(torch.device('cuda:0'))
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0
1.13.0
cuda


In [1]:
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
!unzip -q nature_12K.zip

--2023-04-07 09:25:24--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 64.233.182.128, 173.194.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   208MB/s    in 17s     

2023-04-07 09:25:40 (221 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [8]:
# imports
import math
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader, random_split
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

from torchvision.datasets import ImageFolder
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pathlib
 

# Dataset Augmentation 
def load_data(bs,augment_data=False):
    # define the transforms to be applied to the training data
    if augment_data:
        train_transforms = transforms.Compose([
          transforms.Resize((300,300)),
          transforms.RandomHorizontalFlip(),
          transforms.RandomRotation(10),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])
    else:
        train_transforms = transforms.Compose([
          transforms.Resize((300,300)),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])
    
    test_transforms = transforms.Compose([
      transforms.Resize((300,300)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])


    home_path = "/kaggle/working/inaturalist_12K"

    train_path = os.path.join(home_path,'train')
    test_path = os.path.join(home_path,'val')
    # define the dataset and apply the transforms
    train_dataset = ImageFolder(train_path, transform=train_transforms)
    test_dataset = ImageFolder(test_path, transform=test_transforms)

    # split training dataset into train and validation sets
    train_size = int(0.8 * len(train_dataset))
    print(train_size)
    val_size = len(train_dataset) - train_size
    print(val_size)

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # create a data loader for the training data
    train_loader = torch.utils.data.DataLoader(train_dataset,bs, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, bs, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, bs, shuffle=False)

    #categories
    root=pathlib.Path(train_path)

    classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

    return train_loader,val_loader,test_loader,classes





In [9]:
#Simple CNN
def flatten(k=[11,9,7,5,3],w=300, s=1, p=1):
    r=w
    for i in  range(len(k)):
        print("r",r)
        r= (r+2*p-k[i])+1
        r= int(r/2)+1
    return r 


class CNN(nn.Module):
    def __init__(self, in_channels=3, num_class=10,num_filters=4,kernel_sizes=[11,9,7,5,3],fc_neurons=64,batch_norm=True,dropout=0.3,filter_multiplier=2,activation='LeakyRelu'):

        super(CNN, self).__init__()
        self.in_channels=in_channels
        self.num_class=num_class
        self.num_filters=num_filters
        self.kernel_sizes=kernel_sizes
        self.fc_neurons=fc_neurons
        self.activation=activation
        self.batch_norm=batch_norm
        self.dropout=dropout
        self.filter_multiplier=filter_multiplier
        
        #print("in1")
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=kernel_sizes[0],stride=1, padding=1).to(device)
        #print("in2")
        self.bn1=nn.BatchNorm2d(num_features=num_filters)
        self.relu1 = nn.LeakyReLU()

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv2 = nn.Conv2d(num_filters,filter_multiplier*num_filters,  kernel_size=kernel_sizes[1],stride=1, padding=1).to(device)
        self.bn2=nn.BatchNorm2d(num_features=filter_multiplier*num_filters)
        self.relu2 = nn.LeakyReLU()

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv3 = nn.Conv2d(filter_multiplier*num_filters, int(math.pow(filter_multiplier, 2))*num_filters, kernel_size=kernel_sizes[2],stride=1, padding=1).to(device)
        self.bn3=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 2))*num_filters)
        self.relu3 = nn.LeakyReLU()


        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv4 = nn.Conv2d(int(math.pow(filter_multiplier, 2))*num_filters, int(math.pow(filter_multiplier, 3))*num_filters, kernel_size=kernel_sizes[3],stride=1, padding=1).to(device)
        self.bn4=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 3))*num_filters)
        self.relu4 = nn.LeakyReLU()

        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv5 = nn.Conv2d(int(math.pow(filter_multiplier, 3))*num_filters, int(math.pow(filter_multiplier, 4))*num_filters, kernel_size=kernel_sizes[4],stride=1, padding=1).to(device)
        self.bn5=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 4))*num_filters)
        self.relu5 = nn.LeakyReLU()


        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        print("ok pool5")
        self.r=flatten(kernel_sizes)
        print("ok flatten")
        print(self.r)
        self.fc1 = nn.Linear(in_features=int(math.pow(filter_multiplier, 4))*num_filters*self.r*self.r, out_features=fc_neurons)
        self.relu6 = nn.LeakyReLU()

        self.drop = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(in_features=fc_neurons, out_features=num_class)
    
    def forward(self, x):
        x = self.conv1(x)
        if self.batch_norm:
            x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.pool1(x)
        
        x = self.conv2(x)
        if self.batch_norm:
            x = self.bn2(x)
        x = self.relu2(x)

        x = self.pool2(x)

        x = self.conv3(x)
        if self.batch_norm:
            x = self.bn3(x)
        x = self.relu3(x)
        
        x = self.pool3(x)
        
        x = self.conv4(x)
        if self.batch_norm:
            x = self.bn4(x)
        x = self.relu4(x)
       
        x = self.pool4(x)
     
        x = self.conv5(x)
        if self.batch_norm:
            x = self.bn5(x)
        x = self.relu5(x)

        x = self.pool5(x)
        x = x.view(x.size(0),-1)

        x = self.fc1(x)
        x = self.relu6(x)
        x = self.drop(x)
        
        x = self.fc2(x)
        
        return x





In [12]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 3
num_class = 10
learning_rate = 0.0001
batch_size = 128
epochs = 10

# Load data
train_loader,val_loader,test_loader,classes=load_data(batch_size)
print(classes)
trainfeature, trainlabel = next(iter(train_loader))
print(f"Feature Batch Shape: {trainfeature.size()}")
print(f"Label Batch Shape: {trainlabel.size()}")



# Initialize network
model = CNN(3,10,4,[11,9,7,5,3],64,True,0.1,2,'LeakyRelu').to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=0.0001)

# Train Network
for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        optimizer.zero_grad()
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
    # Set the model to evaluation mode
    model.eval()

    # Track the total loss and number of correct predictions
    val_loss = 0
    num_correct = 0
    num_samples = 0

    # Evaluate the model on the validation set
    with torch.no_grad():
        for data, targets in val_loader:
            data = data.to(device=device)
            targets = targets.to(device=device)

            scores = model(data)
            val_loss += criterion(scores, targets).item()

            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    # Calculate the average validation loss and accuracy
    val_loss /= len(val_loader)
    val_acc = float(num_correct) / num_samples

    # Print the epoch number, loss, and accuracy
    print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.2f}%'
          .format(epoch+1, epochs, loss.item(), val_loss, val_acc*100))

# Check accuracy on training & test to see how good our model


7999
2000
['.DS_Store', 'Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Feature Batch Shape: torch.Size([128, 3, 300, 300])
Label Batch Shape: torch.Size([128])
ok pool5
r 300
r 147
r 71
r 34
r 17
ok flatten
9
Epoch [1/10], Train Loss: 2.1579, Val Loss: 2.1032, Val Acc: 25.00%


KeyboardInterrupt: 

In [10]:
from signal import signal,SIGPIPE, SIG_DFL
signal(SIGPIPE,SIG_DFL)
!pip install wandb -qU
import wandb
!wandb login da816d14625ef44d200ee4acaa517646962e6f9a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:

sweep_config = {
    "name" : "CS6910_Assignment_2_Q2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "epochs" : {
            "values" : [10,15,20]
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        'learning_rate':{
            "values": [0.001,0.0001,0.0003,0.0005]
        },
        "dropout": {
            "values": [0,0.2,0.3]
        },
        "batch_norm": {
              "values": [True,False]
        },
        "data_aug": {
              "values": [True,False]
        },
        'kernel_sizes':{
            'values': [[3,3,3,3,3],[5,5,5,5,5],[7,5,5,3,3], [11,9,7,5,3]]
        },
        'filter_multiplier': {
            'values': [1, 2, 0.5]
        },
        'num_filters': {
            'values': [4,8,16]
        },
        "fc_neurons": {
              "values": [32, 64, 128]
          }        
    }
}

def calculate_accuracy(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def train():
    config_default={
      'epochs':15,
      'batch_size':32,
      'learning_rate':0.001,
      'dropout':0.3,
      'batch_norm':True,
      'data_aug':True,
      'kernel_sizes':[5,5,5,5,5],
      'filter_multiplier': 2,
      'num_filters': 16,
      "fc_neurons": 64
  }
    wandb.init(config=config_default)
    c= wandb.config
    name = "nfliter_"+str(c.num_filters)
    wandb.init(name=name)

    # Retrieve the hyperparameters from the config
    lr = c.learning_rate
    bs = c.batch_size
    epochs = c.epochs
    dp = c.dropout
    bn = c.batch_norm
    da=c.data_aug
    ks=c.kernel_sizes
    fm=c.filter_multiplier
    nf=c.num_filters
    fc=c.fc_neurons


    # Load the dataset
    train_loader,val_loader,test_loader,classes=load_data(bs,da)
    
    print("data loaded ====================================================")

    # Initialize network
    model = CNN(3,10,nf,ks,fc,bn,dp,fm,'LeakyRelu').to(device)
    print("model ini==============================================================")
    # Loss and optimizer
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=0.0001)

    # Train Network
    for epoch in range(epochs):
        # Set the model to training mode
        model.train()

        for batch_idx, (data, targets) in enumerate(train_loader):
            # Get data to cuda if possible
            data = data.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward

            loss.backward()

            # gradient descent or adam step
            optimizer.step()          
        # Calculate the test accuracy
        train_acc = calculate_accuracy(model, train_loader)
        val_acc = calculate_accuracy(model, val_loader)
        test_acc = calculate_accuracy(model, test_loader)

        # Log the metrics to WandB
        wandb.log({'epoch': epoch+1, 'loss': loss.item(), 'test_acc': test_acc,'train_acc': train_acc,'val_acc': val_acc})


    # Save the best model
    wandb.save('model.pt')
    return


In [ ]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment_2_Q2')
wandb.agent(sweep_id, function=train,count=5)


Create sweep with ID: pcyv8fru
Sweep URL: https://wandb.ai/cs22s015/CS6910_Assignment_2_Q2/sweeps/pcyv8fru


wandb: Waiting for W&B process to finish... (success).
wandb: 🚀 View run effortless-sweep-4 at: https://wandb.ai/cs22s015/CS6910_Assignment_2_Q2/runs/7863emce
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230407_103201-7863emce/logs
wandb: Agent Starting Run: wsw03q65 with config:
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_aug: False
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	fc_neurons: 128
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [5, 5, 5, 5, 5]
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 16


7999
2000
data loaded ====================================================
ok pool5
r 300
r 150
r 75
r 37
r 18
ok flatten
9
model ini==============================================================
